<a href="https://colab.research.google.com/github/tylerlum/ufc_automated_scoring_system/blob/main/UFC_Automated_Scoring_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFC Automated Scoring System

The goal of this notebook is to:
* Explore the FightMetrics webpage to scrape the fight and fighter information we need
* Store the fight and fighter data into csv files
* Preprocess the data
* Train and evaluate a neural network to predict fight outcomes

## All bouts and all fighters

In [20]:
all_past_events_url = "http://ufcstats.com/statistics/events/completed"
all_past_events_tables = pd.read_html(all_past_events_url)[0] # Returns list of all tables on page

In [21]:
all_past_events_tables.head()

,Name/date,Location
0,NaN,NaN
1,"UFC Fight Night: Hall vs. Silva October 31, 2020","Las Vegas, Nevada, USA"
2,"UFC 254: Khabib vs. Gaethje October 24, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates"
3,UFC Fight Night: Ortega vs. The Korean Zombie ...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"
4,UFC Fight Night: Moraes vs. Sandhagen October...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"


In [57]:
from string import ascii_lowercase

all_fighters_tables = []
for c in ascii_lowercase:
  all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
  all_fighters_table = pd.read_html(all_fighters_url)[0]
  all_fighters_tables.append(all_fighters_table)

all_fighters_tables = pd.concat(all_fighters_tables)

In [58]:
all_fighters_tables.head()

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN


## Getting the data for one fight

This will look at Brian Ortega vs. Chan Sung Jung.

UFC Fight Night: Ortega vs. The Korean Zombie

http://ufcstats.com/fight-details/5a440bdee19efaab

In [1]:
import pandas as pd

In [8]:
ortega_vs_jung_url = "http://ufcstats.com/fight-details/5a440bdee19efaab"
ortega_vs_jung_tables = pd.read_html(ortega_vs_jung_url)
print(f"This URL has {len(ortega_vs_jung_tables)} tables")

This URL has 4 tables


In [11]:
ortega_vs_jung_tables[0].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Brian Ortega Chan Sung Jung,2 0,127 of 212 62 of 163,59% 38%,129 of 214 64 of 166,3 of 10 0 of 0,30% ---,0 0,0 0,0:57 0:00


In [46]:
ortega_vs_jung_tables[1].head()

Fighter      KD  ...    Rev.        Ctrl
                       Round 1 Round 1  ... Round 1     Round 1
                       Round 2 Round 2  ... Round 2     Round 2
                       Round 3 Round 3  ... Round 3     Round 3
                       Round 4 Round 4  ... Round 4     Round 4
                       Round 5 Round 5  ... Round 5     Round 5
0  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:00  0:00
1  Brian Ortega Chan Sung Jung    1  0  ...    0  0  0:35  0:00
2  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00
3  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:22  0:00
4  Brian Ortega Chan Sung Jung    0  0  ...    0  0  0:00  0:00

[5 rows x 10 columns]

In [14]:
ortega_vs_jung_tables[2].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Brian Ortega Chan Sung Jung,127 of 212 62 of 163,59% 38%,64 of 135 35 of 127,22 of 28 14 of 21,41 of 49 13 of 15,118 of 199 62 of 162,5 of 7 0 of 1,4 of 6 0 of 0


In [13]:
ortega_vs_jung_tables[3].head()

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground,Unnamed: 9_level_0
,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1,Round 1
,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2,Round 2
,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3,Round 3
,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4,Round 4
,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5,Round 5
0,Brian Ortega Chan Sung Jung,24 of 39 14 of 26,61% 53%,11 of 24 4 of 15,5 of 6 4 of 5,8 of 9 6 of 6,24 of 39 14 of 26,0 of 0 0 of 0,0 of 0 0 of 0,NaN
1,Brian Ortega Chan Sung Jung,23 of 37 14 of 34,62% 41%,14 of 25 12 of 31,2 of 4 0 of 1,7 of 8 2 of 2,17 of 29 14 of 34,2 of 2 0 of 0,4 of 6 0 of 0,NaN
2,Brian Ortega Chan Sung Jung,28 of 42 7 of 32,66% 21%,13 of 25 2 of 23,4 of 4 1 of 3,11 of 13 4 of 6,27 of 41 7 of 32,1 of 1 0 of 0,0 of 0 0 of 0,NaN
3,Brian Ortega Chan Sung Jung,17 of 35 8 of 26,48% 30%,5 of 18 1 of 18,5 of 6 6 of 7,7 of 11 1 of 1,15 of 33 8 of 26,2 of 2 0 of 0,0 of 0 0 of 0,NaN
4,Brian Ortega Chan Sung Jung,35 of 59 19 of 45,59% 42%,21 of 43 16 of 40,6 of 8 3 of 5,8 of 8 0 of 0,35 of 57 19 of 44,0 of 2 0 of 1,0 of 0 0 of 0,NaN


Note how each cell actually has two datapoints, which is a bit frustrating. We need to break them up into different cells.

In [19]:
ortega_vs_jung_tables[0].head()

,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td,Td %,Sub. att,Rev.,Ctrl
0,Brian Ortega Chan Sung Jung,2 0,127 of 212 62 of 163,59% 38%,129 of 214 64 of 166,3 of 10 0 of 0,30% ---,0 0,0 0,0:57 0:00


In [64]:
ortega_vs_jung_tables[0]["Fighter"][0]

'Brian Ortega Chan Sung Jung'

In [73]:
for i, row in all_fighters_tables.iterrows():
    fighter_name = f'{row["First"]} {row["Last"]}'
    if ortega_vs_jung_tables[0]["Fighter"][0].startswith(fighter_name):
        first_fighter = fighter_name
        second_fighter = ortega_vs_jung_tables[0]["Fighter"][0][len(fighter_name)+1:]
        print(f"First fighter is {first_fighter}")
        print(f'Second fighter is {second_fighter}')
        break

First fighter is Brian Ortega
Second fighter is Chan Sung Jung


In [75]:
ortega_vs_jung_tables[0]["KD"][0]

'2  0'

In [81]:
kd_string_split = ortega_vs_jung_tables[0]["KD"][0].split(" ")
first_fighter_kd = " ".join(kd_string_split[:len(kd_string_split)//2])
second_fighter_kd = " ".join(kd_string_split[len(kd_string_split)//2+1:])
print(f"First fighter kds: {first_fighter_kd}")
print(f'Second fighter kds: {second_fighter_kd}')

First fighter kds: 2
Second fighter kds: 0


In [87]:
def parse_two_fighter_stats(fight_table, column):
    string_split = fight_table[column][0].split(" ")
    first_fighter_stat = " ".join(string_split[:len(string_split)//2])
    second_fighter_stat = " ".join(string_split[len(string_split)//2+1:])
    return first_fighter_stat, second_fighter_stat

In [90]:
for column in ortega_vs_jung_tables[0].columns:
    fighter_1_stat, fighter_2_stat = parse_two_fighter_stats(ortega_vs_jung_tables[0], column)
    print(f"{column} 1: {fighter_1_stat} - {column} 2: {fighter_2_stat}")

Fighter 1: Brian Ortega - Fighter 2: Sung Jung
KD 1: 2 - KD 2: 0
Sig. str. 1: 127 of 212 - Sig. str. 2: 62 of 163
Sig. str. % 1: 59% - Sig. str. % 2: 38%
Total str. 1: 129 of 214 - Total str. 2: 64 of 166
Td 1: 3 of 10 - Td 2: 0 of 0
Td % 1: 30% - Td % 2: ---
Sub. att 1: 0 - Sub. att 2: 0
Rev. 1: 0 - Rev. 2: 0
Ctrl 1: 0:57 - Ctrl 2: 0:00


In [77]:
ortega_vs_jung_tables[0]["Sig. str."][0].split(' ')

['127', 'of', '212', '', '62', 'of', '163']

In [ ]:
from urllib.request import urlopen

In [ ]:
url = "http://ufcstats.com/fight-details/5a440bdee19efaab"

In [ ]:
page = urlopen(url)

In [ ]:
page

In [ ]:
html_bytes = page.read()

In [ ]:
html = html_bytes.decode("utf-8")

In [ ]:
html

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <title>\n    Stats | UFC\n  </title>\n  <meta name="description" content="">\n  <meta name="viewport" content="">\n  <link rel="stylesheet" href="/blocks/main.css?ver=612006">\n  <script src="/js/vendor/modernizr-2.6.2.min.js"></script>\n  <script>\n    (function(i,s,o,g,r,a,m){i[\'GoogleAnalyticsObject\']=r;i[r]=i[r]||function(){\n    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),\n    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n    })(window,document,\'script\',\'//www.google-analytics.com/analytics.js\',\'ga\');\n\n    ga(\'creat

In [ ]:
start_index = html.find("<title>") + len("<title>")
end_index = html.find("</title>")

In [ ]:
html_title = html[start_index:end_index]

In [ ]:
html_title

'\n    Stats | UFC\n  '

In [ ]:
html[html.find("Method"):]

'Method:\n\n        </i>\n\n       <i style="font-style: normal"> Decision - Unanimous </i>\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Round:\n        </i>\n        5\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Time:\n        </i>\n        5:00\n\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Time format:\n        </i>\n        5 Rnd (5-5-5-5-5)\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Referee:\n        </i>\n                            <span class="">\n                                Lukasz Bosacki\n                            </span>\n      </i>\n    </p>\n    <p class="b-fight-details__text">\n      <i class="b-fight-details__text-item_first">\n        <i class="b-fight-details__label">\n          Details:\n   

In [ ]:
html[html.find("Round:"):]

'Round:\n        </i>\n        5\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Time:\n        </i>\n        5:00\n\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Time format:\n        </i>\n        5 Rnd (5-5-5-5-5)\n      </i>\n      <i class="b-fight-details__text-item">\n        <i class="b-fight-details__label">\n          Referee:\n        </i>\n                            <span class="">\n                                Lukasz Bosacki\n                            </span>\n      </i>\n    </p>\n    <p class="b-fight-details__text">\n      <i class="b-fight-details__text-item_first">\n        <i class="b-fight-details__label">\n          Details:\n        </i>\n      </i>\n      \n      \n      \n\n          \n\n\n          \n      \n\n          \n\n\n          \n      \n\n          \n\n\n          \n      \n\n          \n\n\n          \n      \n\n   

In [ ]:
html[html.find("Round:"):][:100]

'Round:\n        </i>\n        5\n      </i>\n      <i class="b-fight-details__text-item">\n        <i cla'

In [ ]:
helper = html[html.find("Round:"):][:100]

In [ ]:
index_of_round = [s.isnumeric() for s in helper].index(True)
round = int(helper[index_of_round])

In [ ]:
round

5

In [ ]:
if round > 1:
  average_length = html.find("Round 2") - html.find("Round 1")
else:
  average_length = 1000
all_round_info = html[html.find("Round 1"):html.find(f"Round {round}")+average_length*2]

In [ ]:
average_length

2338

In [ ]:
all_round_info

'Round 1\n            </th>\n          </thead>\n          \n            <tr class="b-fight-details__table-row">\n  <td  class="b-fight-details__table-col l-page_align_left">\n    <p class="b-fight-details__table-text">\n      <a class=\'b-link b-link_style_black\' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>\n    </p>\n    <p class="b-fight-details__table-text">\n      <a class=\'b-link b-link_style_black\' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>\n    </p>\n  </td>\n  <td  class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      1\n    </p>\n    <p class="b-fight-details__table-text">\n      0\n    </p>\n  </td>\n  <td  class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      24 of 39\n    </p>\n    <p class="b-fight-details__table-text">\n      14 of 26\n    </p>\n  </td>\n  <td   class="b-fight-details__table-col">\n    <p class="b-fight-details__table-t

In [ ]:
all_round_info[all_round_info.index(" of ")-10:]

'>\n      24 of 39\n    </p>\n    <p class="b-fight-details__table-text">\n      14 of 26\n    </p>\n  </td>\n  <td   class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      61%\n    </p>\n    <p class="b-fight-details__table-text">\n      53%\n    </p>\n  </td>\n  <td  class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      24 of 39\n    </p>\n    <p class="b-fight-details__table-text">\n      14 of 26\n    </p>\n  </td>\n  <td  class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      0 of 0\n    </p>\n    <p class="b-fight-details__table-text">\n      0 of 0\n    </p>\n  </td>\n  <td    class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      ---\n    </p>\n    <p class="b-fight-details__table-text">\n      ---\n    </p>\n  </td>\n  <td    class="b-fight-details__table-col">\n    <p class="b-fight-details__table-text">\n      0\n    </p>\n    <p class="b-fight

In [ ]:
Round 1
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 39
    </p>
    <p class="b-fight-details__table-text">
      14 of 26
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      61%
    </p>
    <p class="b-fight-details__table-text">
      53%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 39
    </p>
    <p class="b-fight-details__table-text">
      14 of 26
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      ---
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:00
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 2
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      23 of 37
    </p>
    <p class="b-fight-details__table-text">
      14 of 34
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      62%
    </p>
    <p class="b-fight-details__table-text">
      41%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 38
    </p>
    <p class="b-fight-details__table-text">
      14 of 34
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      2 of 3
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      66%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:35
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 3
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      28 of 42
    </p>
    <p class="b-fight-details__table-text">
      7 of 32
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      66%
    </p>
    <p class="b-fight-details__table-text">
      21%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      28 of 42
    </p>
    <p class="b-fight-details__table-text">
      7 of 32
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 1
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:00
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 4
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      17 of 35
    </p>
    <p class="b-fight-details__table-text">
      8 of 26
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      48%
    </p>
    <p class="b-fight-details__table-text">
      30%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      18 of 36
    </p>
    <p class="b-fight-details__table-text">
      10 of 29
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1 of 4
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      25%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:22
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 5
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      35 of 59
    </p>
    <p class="b-fight-details__table-text">
      19 of 45
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      59%
    </p>
    <p class="b-fight-details__table-text">
      42%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      35 of 59
    </p>
    <p class="b-fight-details__table-text">
      19 of 45
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 2
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:00
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      

      </tbody>
    </table>
    </section>
<section class="b-fight-details__section js-fight-section">
      <p class="b-fight-details__collapse-link_tot" style="margin-bottom: 0px;margin-top: 0px;">
        Significant Strikes

      </p>

    </section>

    <table style="width: 745px"  >
      <thead class="b-fight-details__table-head">
      <tr class="b-fight-details__table-row">
        <th class="b-fight-details__table-col l-page_align_left">
          Fighter
        </th>
        <th class="b-fight-details__table-col">
          Sig. str
        </th>
        <th class="b-fight-details__table-col">
          Sig. str. %
        </th>
        <th class="b-fight-details__table-col">
          Head
        </th>
        <th class="b-fight-details__table-col">
          Body
        </th>
        <th class="b-fight-details__table-col">
          Leg
        </th>
        <th class="b-fight-details__table-col">
          Distance
        </th>
        <th class="b-fight-details__table-col">
          Clinch
        </th>
        <th class="b-fight-details__table-col">
          Ground
        </th>
      </tr>
      </thead>
      <tbody class="b-fight-details__table-body">
      <tr class="b-fight-details__table-row">
  <td class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      127 of 212
    </p>
    <p class="b-fight-details__table-text">
      62 of 163
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      59%
    </p>
    <p class="b-fight-details__table-text">
      38%
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      64 of 135
    </p>
    <p class="b-fight-details__table-text">
      35 of 127
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      22 of 28
    </p>
    <p class="b-fight-details__table-text">
      14 of 21
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      41 of 49
    </p>
    <

In [ ]:
<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js"> <!--<![endif]-->
<head>
  <meta charset="utf-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge">
  <title>
    Stats | UFC
  </title>
  <meta name="description" content="">
  <meta name="viewport" content="">
  <link rel="stylesheet" href="/blocks/main.css?ver=612006">
  <script src="/js/vendor/modernizr-2.6.2.min.js"></script>
  <script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send', 'pageview');
  </script>
</head>
<body class="b-page" data-link="home">
    <header class="b-head">
      <div class="b-head__nav-bar">
  <div class="l-page__container">
    <a class="b-head__logo" href="/">
        <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ufc-stats-logo.png"
             alt="">
    </a>
    <menu class="b-head__menu">
      
          <li class="b-head__item b-head__item_state_current">
            <a class="b-head__menu-link" href="/statistics/events/">
              EVENTS & FIGHTS
            </a>
          </li> 
      
      
          <li class="b-head__item">
            <a class="b-head__menu-link" href="/statistics/fighters">
              FIGHTERS
            </a>
          </li>
      
      
          <li class="b-head__item">
            <a class="b-head__menu-link" href="http://statleaders.ufc.com" target="_blank">
              STAT LEADERS
            </a>
          </li>
      
    </menu>
  </div>
</div>
    <div class="b-head__sub-nav">
    </div>

    </header>
    <section class="b-statistics__section_details">
    <div class="l-page__container">
        
            <h2 class="b-content__title">
  <a href="http://ufcstats.com/event-details/d4f364dd076bb0e2"  class="b-link">

  UFC Fight Night: Ortega vs. The Korean Zombie

   </a>
</h2>
<div class="b-fight-details">
<div class="b-fight-details__persons clearfix">
  
<div class="b-fight-details__person">
  <i class="b-fight-details__person-status b-fight-details__person-status_style_green">
    W
  </i>
  <div class="b-fight-details__person-text">
    <h3 class="b-fight-details__person-name">
      <a class='b-link b-fight-details__person-link' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </h3>
    <p class="b-fight-details__person-title">
      "T-City"
    </p>
  </div>
</div>
  
<div class="b-fight-details__person">
  <i class="b-fight-details__person-status b-fight-details__person-status_style_gray">
    L
  </i>
  <div class="b-fight-details__person-text">
    <h3 class="b-fight-details__person-name">
      <a class='b-link b-fight-details__person-link' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </h3>
    <p class="b-fight-details__person-title">
      "The Korean Zombie"
    </p>
  </div>
</div>
</div>
<div class="b-fight-details__fight">
  <div class="b-fight-details__fight-head">
    <i class="b-fight-details__fight-title">
      
      Featherweight Bout
    </i>
  </div>
  <div class="b-fight-details__content">
    <p class="b-fight-details__text">
      <i class="b-fight-details__text-item_first">
        <i class="b-fight-details__label">
          Method:

        </i>

       <i style="font-style: normal"> Decision - Unanimous </i>
      </i>
      <i class="b-fight-details__text-item">
        <i class="b-fight-details__label">
          Round:
        </i>
        5
      </i>
      <i class="b-fight-details__text-item">
        <i class="b-fight-details__label">
          Time:
        </i>
        5:00

      </i>
      <i class="b-fight-details__text-item">
        <i class="b-fight-details__label">
          Time format:
        </i>
        5 Rnd (5-5-5-5-5)
      </i>
      <i class="b-fight-details__text-item">
        <i class="b-fight-details__label">
          Referee:
        </i>
                            <span class="">
                                Lukasz Bosacki
                            </span>
      </i>
    </p>
    <p class="b-fight-details__text">
      <i class="b-fight-details__text-item_first">
        <i class="b-fight-details__label">
          Details:
        </i>
      </i>
      
      
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      

          


          
      
      
       
      
          <i class="b-fight-details__text-item">
          

            <span>Ben Cartlidge</span> 

            
            
            45 - 50.
          </i>
      
          <i class="b-fight-details__text-item">
          

            <span>David Lethaby</span> 

            
            
            45 - 50.
          </i>
      
          <i class="b-fight-details__text-item">
          

            <span>Clemens Werner</span> 

            
            
            45 - 50.
          </i>
      

    </p>
  </div>
</div>

 <section class="b-fight-details__section js-fight-section">
      <p class="b-fight-details__collapse-link_tot ">
        Totals
     </p>
    </section>
    <section class="b-fight-details__section js-fight-section">
    <table style="width: 745px"  >
      <thead class="b-fight-details__table-head">
      <tr class="b-fight-details__table-row">
        <th class="b-fight-details__table-col l-page_align_left">
          Fighter
        </th>
        <th class="b-fight-details__table-col">
          KD
        </th>
        <th class="b-fight-details__table-col">
          Sig. str.
        </th>
        <th class="b-fight-details__table-col">
          Sig. str. %
        </th>
        <th class="b-fight-details__table-col">
          Total str.
        </th>
        <th class="b-fight-details__table-col">
          Td
        </th>
        <th class="b-fight-details__table-col">
          Td %
        </th>
        <th class="b-fight-details__table-col">
          Sub. att
        </th>
        <th class="b-fight-details__table-col">
          Rev.
        </th>
        <th class="b-fight-details__table-col">
          Ctrl
        </th>
      </tr>
      </thead>
      <tbody class="b-fight-details__table-body">
      <tr class="b-fight-details__table-row">
  <td class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      2
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      127 of 212
    </p>
    <p class="b-fight-details__table-text">
      62 of 163
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      59%
    </p>
    <p class="b-fight-details__table-text">
      38%
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      129 of 214
    </p>
    <p class="b-fight-details__table-text">
      64 of 166
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      3 of 10
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      30%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:57
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>

      </tbody>
      </table>
    </section>
    <section class="b-fight-details__section js-fight-section">
    <a class="b-fight-details__collapse-link_rnd js-fight-collapse-link">

      <i class="b-fight-details__collapse-left">
        Per round
        <i class="b-fight-details__arrow"></i>
      </i>
    </a>
    <table class="b-fight-details__table js-fight-table"  style="width: 745px;margin-bottom: 0px;">
      <thead class="b-fight-details__table-head_rnd" >
      <tr  class="b-fight-details__table-row">
        <th class="b-fight-details__table-col l-page_align_left">
          Fighter
        </th>
        <th class="b-fight-details__table-col">
          KD
        </th>
        <th class="b-fight-details__table-col">
          Sig. str.
        </th>
        <th class="b-fight-details__table-col">
          Sig. str. %
        </th>
        <th class="b-fight-details__table-col">
          Total str.
        </th>
        <th class="b-fight-details__table-col">
          Td %
        </th>
        <th class="b-fight-details__table-col">
          Td %
        </th>
        <th class="b-fight-details__table-col">
          Sub. att
        </th>
        <th class="b-fight-details__table-col">
          Rev.
        </th>
        <th class="b-fight-details__table-col">
          Ctrl
        </th>
      </tr>
      </thead>
      <tbody   class="b-fight-details__table-body">
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 1
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 39
    </p>
    <p class="b-fight-details__table-text">
      14 of 26
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      61%
    </p>
    <p class="b-fight-details__table-text">
      53%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 39
    </p>
    <p class="b-fight-details__table-text">
      14 of 26
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      ---
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:00
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 2
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      23 of 37
    </p>
    <p class="b-fight-details__table-text">
      14 of 34
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      62%
    </p>
    <p class="b-fight-details__table-text">
      41%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 38
    </p>
    <p class="b-fight-details__table-text">
      14 of 34
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      2 of 3
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      66%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:35
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 3
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      28 of 42
    </p>
    <p class="b-fight-details__table-text">
      7 of 32
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      66%
    </p>
    <p class="b-fight-details__table-text">
      21%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      28 of 42
    </p>
    <p class="b-fight-details__table-text">
      7 of 32
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 1
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:00
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 4
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      17 of 35
    </p>
    <p class="b-fight-details__table-text">
      8 of 26
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      48%
    </p>
    <p class="b-fight-details__table-text">
      30%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      18 of 36
    </p>
    <p class="b-fight-details__table-text">
      10 of 29
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1 of 4
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      25%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:22
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      



          <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 5
            </th>
          </thead>
          
            <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      35 of 59
    </p>
    <p class="b-fight-details__table-text">
      19 of 45
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      59%
    </p>
    <p class="b-fight-details__table-text">
      42%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      35 of 59
    </p>
    <p class="b-fight-details__table-text">
      19 of 45
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 2
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0%
    </p>
    <p class="b-fight-details__table-text">
      ---
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>  
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0
    </p>
    <p class="b-fight-details__table-text">
      0
    </p>
  </td>
  <td   class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0:00
    </p>
    <p class="b-fight-details__table-text">
      0:00
    </p>
  </td>
</tr>
          
      

      </tbody>
    </table>
    </section>
<section class="b-fight-details__section js-fight-section">
      <p class="b-fight-details__collapse-link_tot" style="margin-bottom: 0px;margin-top: 0px;">
        Significant Strikes

      </p>

    </section>

    <table style="width: 745px"  >
      <thead class="b-fight-details__table-head">
      <tr class="b-fight-details__table-row">
        <th class="b-fight-details__table-col l-page_align_left">
          Fighter
        </th>
        <th class="b-fight-details__table-col">
          Sig. str
        </th>
        <th class="b-fight-details__table-col">
          Sig. str. %
        </th>
        <th class="b-fight-details__table-col">
          Head
        </th>
        <th class="b-fight-details__table-col">
          Body
        </th>
        <th class="b-fight-details__table-col">
          Leg
        </th>
        <th class="b-fight-details__table-col">
          Distance
        </th>
        <th class="b-fight-details__table-col">
          Clinch
        </th>
        <th class="b-fight-details__table-col">
          Ground
        </th>
      </tr>
      </thead>
      <tbody class="b-fight-details__table-body">
      <tr class="b-fight-details__table-row">
  <td class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      127 of 212
    </p>
    <p class="b-fight-details__table-text">
      62 of 163
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      59%
    </p>
    <p class="b-fight-details__table-text">
      38%
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      64 of 135
    </p>
    <p class="b-fight-details__table-text">
      35 of 127
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      22 of 28
    </p>
    <p class="b-fight-details__table-text">
      14 of 21
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      41 of 49
    </p>
    <p class="b-fight-details__table-text">
      13 of 15
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      118 of 199
    </p>
    <p class="b-fight-details__table-text">
      62 of 162
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      5 of 7
    </p>
    <p class="b-fight-details__table-text">
      0 of 1
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      4 of 6
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
</tr>

      </tbody>
    </table>
    <section class="b-fight-details__section js-fight-section">
      <a class="b-fight-details__collapse-link_rnd js-fight-collapse-link">

        <i class="b-fight-details__collapse-left">
          Per round
          <i class="b-fight-details__arrow"></i>
        </i>
      </a>
      <table class="b-fight-details__table js-fight-table"  style="width: 745px;margin-bottom: 0px;">
        <thead class="b-fight-details__table-head_rnd"  >
        <tr class="b-fight-details__table-row">
          <th class="b-fight-details__table-col l-page_align_left">
            Fighter
          </th>
          <th class="b-fight-details__table-col">
            Sig. str
          </th>
          <th class="b-fight-details__table-col">
            Sig. str. %
          </th>
          <th class="b-fight-details__table-col">
            Head
          </th>
          <th class="b-fight-details__table-col">
            Body
          </th>
          <th class="b-fight-details__table-col">
            Leg
          </th>
          <th class="b-fight-details__table-col">
            Distance
          </th>
          <th class="b-fight-details__table-col">
            Clinch
          </th>
          <th class="b-fight-details__table-col">
            Ground
          </th>
        </tr>
        </thead>
        <tbody class="b-fight-details__table-body">
        



            <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 1
            </th>
            </thead>
            
              <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 39
    </p>
    <p class="b-fight-details__table-text">
      14 of 26
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      61%
    </p>
    <p class="b-fight-details__table-text">
      53%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      11 of 24
    </p>
    <p class="b-fight-details__table-text">
      4 of 15
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      5 of 6
    </p>
    <p class="b-fight-details__table-text">
      4 of 5
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      8 of 9
    </p>
    <p class="b-fight-details__table-text">
      6 of 6
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      24 of 39
    </p>
    <p class="b-fight-details__table-text">
      14 of 26
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
</tr>
            
        



            <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 2
            </th>
            </thead>
            
              <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      23 of 37
    </p>
    <p class="b-fight-details__table-text">
      14 of 34
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      62%
    </p>
    <p class="b-fight-details__table-text">
      41%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      14 of 25
    </p>
    <p class="b-fight-details__table-text">
      12 of 31
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      2 of 4
    </p>
    <p class="b-fight-details__table-text">
      0 of 1
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      7 of 8
    </p>
    <p class="b-fight-details__table-text">
      2 of 2
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      17 of 29
    </p>
    <p class="b-fight-details__table-text">
      14 of 34
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      2 of 2
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      4 of 6
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
</tr>
            
        



            <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 3
            </th>
            </thead>
            
              <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      28 of 42
    </p>
    <p class="b-fight-details__table-text">
      7 of 32
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      66%
    </p>
    <p class="b-fight-details__table-text">
      21%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      13 of 25
    </p>
    <p class="b-fight-details__table-text">
      2 of 23
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      4 of 4
    </p>
    <p class="b-fight-details__table-text">
      1 of 3
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      11 of 13
    </p>
    <p class="b-fight-details__table-text">
      4 of 6
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      27 of 41
    </p>
    <p class="b-fight-details__table-text">
      7 of 32
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      1 of 1
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
</tr>
            
        



            <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 4
            </th>
            </thead>
            
              <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      17 of 35
    </p>
    <p class="b-fight-details__table-text">
      8 of 26
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      48%
    </p>
    <p class="b-fight-details__table-text">
      30%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      5 of 18
    </p>
    <p class="b-fight-details__table-text">
      1 of 18
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      5 of 6
    </p>
    <p class="b-fight-details__table-text">
      6 of 7
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      7 of 11
    </p>
    <p class="b-fight-details__table-text">
      1 of 1
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      15 of 33
    </p>
    <p class="b-fight-details__table-text">
      8 of 26
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      2 of 2
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
</tr>
            
        



            <thead class="b-fight-details__table-row b-fight-details__table-row_type_head">

            <th class="b-fight-details__table-col" colspan="10">
              Round 5
            </th>
            </thead>
            
              <tr class="b-fight-details__table-row">
  <td  class="b-fight-details__table-col l-page_align_left">
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/def8166ff24bd237>Brian Ortega </a>
    </p>
    <p class="b-fight-details__table-text">
      <a class='b-link b-link_style_black' href=http://ufcstats.com/fighter-details/c451d67c09c55418>Chan Sung Jung </a>
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      35 of 59
    </p>
    <p class="b-fight-details__table-text">
      19 of 45
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      59%
    </p>
    <p class="b-fight-details__table-text">
      42%
    </p>
  </td>
  <td  class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      21 of 43
    </p>
    <p class="b-fight-details__table-text">
      16 of 40
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      6 of 8
    </p>
    <p class="b-fight-details__table-text">
      3 of 5
    </p>
  </td>
  <td    class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      8 of 8
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      35 of 57
    </p>
    <p class="b-fight-details__table-text">
      19 of 44
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 2
    </p>
    <p class="b-fight-details__table-text">
      0 of 1
    </p>
  </td>
  <td class="b-fight-details__table-col">
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
    <p class="b-fight-details__table-text">
      0 of 0
    </p>
  </td>
</tr>
            
        

        </tbody>
      </table>
    </section>

<section class="b-fight-details__section">
  <div class="b-fight-details__charts">
    <header class="b-fight-details__charts-head b-fight-details__charts-head_style_bordered">
      <h3 class="b-fight-details__charts-head-title">
        Significant Strikes
      </h3>
      <i class="b-fight-details__charts-name b-fight-details__charts-name_pos_left js-chart-name" data-color="red">
        <i class="b-fight-details__charts-bullet"></i>
        Brian Ortega
      </i>
      <i class="b-fight-details__charts-name b-fight-details__charts-name_pos_right js-chart-name" data-color="blue">
        <i class="b-fight-details__charts-bullet"></i>
        Chan Sung Jung
      </i>
    </header>
    
    
    
    
    <div class="b-fight-details__charts-body clearfix">
      <div class="b-fight-details__charts-col">
        <div class="b-fight-details__charts-col-row clearfix">
          <h4 class="b-fight-details__charts-title">
            Landed by target
          </h4>
          <div class="b-fight-details__chart-inner clearfix">
            


<div class="b-fight-details__stacked-chart js-red>">
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_red js-alt" style="height: 50.39370078740158%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_dark-red js-alt" style="height: 17.322834645669293%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_light-red js-alt" style="height: 32.28346456692913%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
</div>
            <div class="b-fight-details__charts-table">
              <div class="b-fight-details__charts-row">
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_red b-fight-details__charts-num_pos_left js-red">
                  50%
                </i>
                <i class="b-fight-details__charts-row-title">
                  Head
                </i>
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_blue b-fight-details__charts-num_pos_right js-blue">
                  56%
                </i>
              </div>
              <div class="b-fight-details__charts-row">
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_dark-red b-fight-details__charts-num_pos_left js-red">
                  17%
                </i>
                <i class="b-fight-details__charts-row-title">
                  Body
                </i>
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_dark-blue b-fight-details__charts-num_pos_right js-blue">
                  22%
                </i>
              </div>
              <div class="b-fight-details__charts-row">
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_light-red b-fight-details__charts-num_pos_left js-red">
                  32%
                </i>
                <i class="b-fight-details__charts-row-title">
                  Leg
                </i>
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_light-blue b-fight-details__charts-num_pos_right js-blue">
                  20%
                </i>
              </div>
            </div>
            


<div class="b-fight-details__stacked-chart js-blue>">
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_blue js-alt" style="height: 56.45161290322581%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_dark-blue js-alt" style="height: 22.580645161290324%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_light-blue js-alt" style="height: 20.967741935483872%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
</div>
          </div>
        </div>
        <div class="b-fight-details__charts-col-row clearfix">
          <h4 class="b-fight-details__charts-title">
            Landed by position
          </h4>
          <div class="b-fight-details__chart-inner">
            


<div class="b-fight-details__stacked-chart js-red>">
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_red js-alt" style="height: 92.91338582677166%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_dark-red js-alt" style="height: 3.937007874015748%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_light-red js-alt" style="height: 3.1496062992125986%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
</div>
            <div class="b-fight-details__charts-table">
              <div class="b-fight-details__charts-row">
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_red b-fight-details__charts-num_pos_left js-red">
                  92%
                </i>
                <i class="b-fight-details__charts-row-title">
                  Distance
                </i>
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_blue b-fight-details__charts-num_pos_right js-blue">
                  100%
                </i>
              </div>
              <div class="b-fight-details__charts-row">
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_dark-red b-fight-details__charts-num_pos_left js-red">
                  3%
                </i>
                <i class="b-fight-details__charts-row-title">
                  Clinch
                </i>
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_dark-blue b-fight-details__charts-num_pos_right js-blue">
                  0%
                </i>
              </div>
              <div class="b-fight-details__charts-row">
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_light-red b-fight-details__charts-num_pos_left js-red">
                  3%
                </i>
                <i class="b-fight-details__charts-row-title">
                  Ground
                </i>
                <i class="b-fight-details__charts-num b-fight-details__charts-num_style_light-blue b-fight-details__charts-num_pos_right js-blue">
                  0%
                </i>
              </div>
            </div>
            


<div class="b-fight-details__stacked-chart js-blue>">
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_blue js-alt" style="height: 100.0%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_dark-blue js-alt" style="height: 0.0%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
  <i class="b-fight-details__stacked-chart-fill b-fight-details__stacked-chart-fill_style_light-blue js-alt" style="height: 0.0%">
    <i class="b-rating__alt b-rating__alt_pos_right-bottom">
      <i class="b-rating__alt-pointer"></i>
      Avg. 50%   High 80%
    </i>
  </i>
</div>
          </div>
        </div>
      </div>
      <div class="b-fight-details__charts-col b-fight-details__charts-col_pos_right">
        <div class="b-fight-details__charts-col-row">
          <h4 class="b-fight-details__charts-title b-fight-details__charts-title_type_left">
            Per round
          </h4>

          <div class="b-fight-details__bar-charts-inner">
            
                
                  <div class="b-fight-details__bar-charts-row clearfix">
  <i class="b-fight-details__bar-chart-title">
    RD1
  </i>
  <i class="b-fight-details__bar-chart-inner">
    <i class="b-fight-details__bar-chart-row js-red">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_red" style="width: 78.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 48.0012px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-red">
        24 of 39 - 61%
      </i>
    </i>
    <i class="b-fight-details__bar-chart-row js-blue">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_blue" style="width: 52.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 28.002px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-blue">
        14 of 26 - 53%
      </i>
    </i>
  </i>
</div>
                
            
                
                  <div class="b-fight-details__bar-charts-row clearfix">
  <i class="b-fight-details__bar-chart-title">
    RD2
  </i>
  <i class="b-fight-details__bar-chart-inner">
    <i class="b-fight-details__bar-chart-row js-red">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_red" style="width: 74.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 45.9984px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-red">
        23 of 37 - 62%
      </i>
    </i>
    <i class="b-fight-details__bar-chart-row js-blue">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_blue" style="width: 68.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 28.0024px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-blue">
        14 of 34 - 41%
      </i>
    </i>
  </i>
</div>
                
            
                
                  <div class="b-fight-details__bar-charts-row clearfix">
  <i class="b-fight-details__bar-chart-title">
    RD3
  </i>
  <i class="b-fight-details__bar-chart-inner">
    <i class="b-fight-details__bar-chart-row js-red">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_red" style="width: 84.00000000000001px">
        <i class="b-fight-details__bar-chart-fill" style="width: 56.002800000000015px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-red">
        28 of 42 - 66%
      </i>
    </i>
    <i class="b-fight-details__bar-chart-row js-blue">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_blue" style="width: 64.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 14.0032px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-blue">
        7 of 32 - 21%
      </i>
    </i>
  </i>
</div>
                
            
                
                  <div class="b-fight-details__bar-charts-row clearfix">
  <i class="b-fight-details__bar-chart-title">
    RD4
  </i>
  <i class="b-fight-details__bar-chart-inner">
    <i class="b-fight-details__bar-chart-row js-red">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_red" style="width: 70.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 33.999px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-red">
        17 of 35 - 48%
      </i>
    </i>
    <i class="b-fight-details__bar-chart-row js-blue">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_blue" style="width: 52.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 16.0004px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-blue">
        8 of 26 - 30%
      </i>
    </i>
  </i>
</div>
                
            
                
                  <div class="b-fight-details__bar-charts-row clearfix">
  <i class="b-fight-details__bar-chart-title">
    RD5
  </i>
  <i class="b-fight-details__bar-chart-inner">
    <i class="b-fight-details__bar-chart-row js-red">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_red" style="width: 118.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 69.99759999999999px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-red">
        35 of 59 - 59%
      </i>
    </i>
    <i class="b-fight-details__bar-chart-row js-blue">
      <i class="b-fight-details__bar-chart b-fight-details__bar-chart_style_blue" style="width: 90.0px">
        <i class="b-fight-details__bar-chart-fill" style="width: 37.998px"></i>
      </i>
      <i class="b-fight-details__bar-chart-text b-fight-details__bar-chart-text_style_light-blue">
        19 of 45 - 42%
      </i>
    </i>
  </i>
</div>
                
            
            <div class="b-fight-details__bar-legend">
              <i class="b-fight-details__bar-legend-row">
                <i class="b-fight-details__bar-legend-dot b-fight-details__bar-legend-dot_style_red js-red"></i>
                <i class="b-fight-details__bar-legend-dot b-fight-details__bar-legend-dot_style_blue js-blue"></i>
                <i class="b-fight-details__bar-legend-text">
                  Landed
                </i>
              </i>
              <i class="b-fight-details__bar-legend-row">
                <i class="b-fight-details__bar-legend-dot b-fight-details__bar-legend-dot_style_red b-fight-details__bar-legend-dot_type_empty js-red"></i>
                <i class="b-fight-details__bar-legend-dot b-fight-details__bar-legend-dot_style_blue b-fight-details__bar-legend-dot_type_empty js-blue"></i>
                <i class="b-fight-details__bar-legend-text">
                  Attempted
                </i>
              </i>
            </div>
          </div>
        </div>
      </div>
      <i class="b-fight-details__charts-line"></i>
    </div>
  </div>
</section>


</div>
        
    </div>
</section>
    <footer class="b-foot">
  <div class="b-foot__inner">
    <div class="l-page__container clearfix">
      <div class="b-foot__right">
        <div class="b-foot__list-inner">
          <h4 class="b-foot__list-title">
            Navigation
          </h4>
          <ul class="b-foot__list">
            <li class="b-foot__item">
              <a class="b-foot__link" href="/statistics/events/">
                Events &amp; Fights
              </a>
            </li>
            <li class="b-foot__item">
              <a class="b-foot__link" href="/statistics/fighters">
                Fighters
              </a>
            </li>
            <li class="b-foot__item">
              <a class="b-foot__link" href="http://statleaders.ufc.com" target="_blank">
                Stat Leaders
              </a>
            </li>
          </ul>
        </div>     
      </div>
      <div class="b-foot__logo">
        <a href="/">
        <img src="http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ufc-stats-foot-logo.png" alt="">
        </a>
        <div class="b-foot__copy-inner">
          &copy; All Rights Reserved |
          <a class="b-foot__copy-link" href="https://ufc.com/terms" target="_blank">Terms of Use</a> |
          <a class="b-foot__copy-link" href="https://ufc.com/privacy-policy" target="_blank">Privacy Policy</a>
        </div>
      </div>      
    </div>
  </div>
</footer>
    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.2/jquery.min.js"></script>
<script src="http://code.jquery.com/jquery-1.9.1.js"></script>
<script src="http://code.jquery.com/ui/1.10.3/jquery-ui.js"></script>
<link rel="stylesheet" href="http://code.jquery.com/ui/1.10.3/themes/smoothness/jquery-ui.css" />
<script>window.jQuery || document.write('<script src="/js/vendor/jquery-1.10.2.min.js"><\/script>')</script>
<script type="text/javascript" src="/js/vendor/jquery.validate.min.js"></script>
<script src="/js/plugins.js"></script>
<script src="/js/main.js?ver=612222"></script>
<script type="text/javascript">
    $(document).ready(function () {
        $(function () {
            $("#dialog").dialog({

                autoOpen: false,
                modal: true,
                show: {
                    effect: "blind",
                    duration: 1000
                },
                hide: {
                    effect: "fold",
                    duration: 1000
                }

            });
        });
    }

    );
    $("#opener").click(function () {
        $("#dialog").dialog("open");
    });
    $("#opener").click(function () {
        var state = true;
        if (state) {
            $("#dialog").animate({
                backgroundColor: "white",
                color: "black"

            }, 1000);
        }
    });

</script>

<script type="text/javascript">

 /*window.onload = function (){
     // location.pathname.split( '/' )
     theUrl = window.location.host;


     if(theUrl === 'hosteddb.fightmetric.com'){
         window.location.replace("http://www.fightmetric.com/statistics/events/upcoming");
     }

     //alert(theUrl);
  }*/
  //window.onpaint = handleHosted;
</script>
</body>
</html>

In [ ]:
import pandas as pd

In [ ]:
tables = pd.read_html(url) # Returns list of all tables on page

In [ ]:
tables[0]["Fighter"][0]

'Brian Ortega Chan Sung Jung'

In [ ]:
all_fighters_url = "http://ufcstats.com/statistics/fighters?char=a&page=all"
all_fighters_tables = pd.read_html(all_fighters_url)
len(all_fighters_tables)

1

In [ ]:
from string import ascii_lowercase

all_fighters_tables = []
for c in ascii_lowercase:
  all_fighters_url = f"http://ufcstats.com/statistics/fighters?char={c}&page=all"
  all_fighters_table = pd.read_html(all_fighters_url)[0]
  all_fighters_tables.append(all_fighters_table)

all_fighters_df = pd.concat(all_fighters_tables)

In [ ]:
all_fighters_df

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
4,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,20.0,5.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
26,Dave,Zitanick,NaN,--,170 lbs.,--,NaN,5.0,7.0,0.0,NaN
27,Alex,Zuniga,NaN,--,145 lbs.,--,NaN,6.0,3.0,0.0,NaN
28,George,Zuniga,NaN,"5' 9""",185 lbs.,--,NaN,3.0,1.0,0.0,NaN
29,Allan,Zuniga,Tigre,"5' 7""",155 lbs.,"70.0""",Orthodox,13.0,1.0,0.0,NaN


In [ ]:
all_fighters_df[all_fighters_df["Last"] == "Jung"]

,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
102,Bu-Kyung,Jung,NaN,"5' 7""",154 lbs.,--,Orthodox,0.0,4.0,0.0,NaN
103,Chan Sung,Jung,The Korean Zombie,"5' 7""",145 lbs.,"72.0""",Orthodox,16.0,6.0,0.0,NaN
104,Young Sam,Jung,NaN,"5' 8""",139 lbs.,--,Orthodox,4.0,8.0,0.0,NaN
105,Da-Un,Jung,Sseda,"6' 4""",205 lbs.,"78.0""",Orthodox,13.0,2.0,0.0,NaN


In [ ]:
tables[0]["Fighter"][0]

'Brian Ortega Chan Sung Jung'

In [ ]:
for first, last in zip(all_fighters_df["First"], all_fighters_df["Last"]):
  name = f"{first} {last}"
  if name in tables[0]["Fighter"][0]:
    print(name)
    break

index_of_name = tables[0]["Fighter"][0].index(name)

Chan Sung Jung


In [ ]:
tables[0]["Fighter"][0]

'Brian Ortega Chan Sung Jung'

In [ ]:
tables[0]["Fighter"][0].replace(name, "")

'Brian Ortega '

In [ ]:
tables[0]["Fighter"][0].replace(name, "").strip()

'Brian Ortega'

In [ ]:
all_past_events_url = "http://ufcstats.com/statistics/events/completed"
all_past_events_tables = pd.read_html(all_past_events_url)[0] # Returns list of all tables on page

In [ ]:
all_past_events_tables

,Name/date,Location
0,NaN,NaN
1,"UFC 254: Khabib vs. Gaethje October 24, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates"
2,UFC Fight Night: Ortega vs. The Korean Zombie ...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"
3,UFC Fight Night: Moraes vs. Sandhagen October...,"Abu Dhabi, Abu Dhabi, United Arab Emirates"
4,"UFC Fight Night: Holm vs. Aldana October 03, ...","Abu Dhabi, Abu Dhabi, United Arab Emirates"
5,"UFC 253: Adesanya vs. Costa September 26, 2020","Abu Dhabi, Abu Dhabi, United Arab Emirates"
6,UFC Fight Night: Covington vs. Woodley Septem...,"Las Vegas, Nevada, USA"
7,UFC Fight Night: Waterson vs. Hill September ...,"Las Vegas, Nevada, USA"
8,UFC Fight Night: Overeem vs. Sakai September ...,"Las Vegas, Nevada, USA"
9,"UFC Fight Night: Smith vs. Rakic August 29, 2020","Las Vegas, Nevada, USA"


In [ ]:
all_past_events_page = urlopen(all_past_events_url)

In [ ]:
all_past_events_html_bytes = all_past_events_page.read()
all_past_events_html = all_past_events_html_bytes.decode("utf-8")

In [ ]:
event1, event2 = all_past_events_tables["Name/date"][1], all_past_events_tables["Name/date"][2]

In [ ]:
i1, i2 = all_past_events_html.find(event1[:event1.find("vs")]), all_past_events_html.find(event2[:event2.find("vs")])
event_string = all_past_events_html[i1:i2]
http_index = event_string.find('http://')
short = event_string[http_index:]
short = short[:short.find("\"")]

In [ ]:
event_string

'UFC 254: Khabib vs. Gaethje\n                        </a>\n                        <span class="b-statistics__date">\n                          October 24, 2020\n                        </span>\n                    </i>\n                  </td>\n                  <td class="b-statistics__table-col b-statistics__table-col_style_big-top-padding">\n                    Abu Dhabi, Abu Dhabi, United Arab Emirates\n                  </td>\n                </tr>\n              \n                <tr class="b-statistics__table-row">\n                  <td class="b-statistics__table-col">\n                  \n                    <i class="b-statistics__table-content">\n                        <a href="http://ufcstats.com/event-details/d4f364dd076bb0e2" class="b-link b-link_style_black">\n                          '

In [ ]:
short

'http://ufcstats.com/event-details/d4f364dd076bb0e2'

In [ ]:
all_past_events_tables["Name/date"][2]

'UFC Fight Night: Ortega vs. The Korean Zombie  October 17, 2020'